In [1]:
from model_v2 import build_lm_classifier_tagger_inference, LSTM_SAVED_STATE
from utils import get_batch_classifier_inference, clean_text_v4 as clean_text
import json
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
with open('109_Bank/checkpoints/model_configs.json', 'r') as inp:
    lm_params = json.load(inp)
with open('109_Bank/checkpoints/classifier_configs.json') as inp:
    cls_params = json.load(inp)
with open('109_Bank/checkpoints/tagger_configs.json') as inp:
    tag_params = json.load(inp)
language_model, classifier, tagger = build_lm_classifier_tagger_inference(lm_params, 
                                                                          cls_params, tag_params, 
                                                                          is_cpu=False)

In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
saver = tf.train.Saver([x for x in tf.global_variables() if x not in tf.get_collection(LSTM_SAVED_STATE)])
saver.restore(session, '109_Bank/checkpoints/chatbot_test/model.cpkt-723')

INFO:tensorflow:Restoring parameters from 109_Bank/checkpoints/chatbot_test/model.cpkt-723


In [5]:
with open('109_Bank/word2idx.json', 'r') as inp:
    word2idx = json.load(inp)
with open('109_Bank/char2idx.json', 'r') as inp:
    char2idx = json.load(inp)
word2char = {w: [char2idx[c] for c in w] for w in word2idx}
with open('Bank/class2idx.json', 'r') as inp:
    class2idx = json.load(inp)
    idx2class = {i: w for w, i in class2idx.items()}
with open('Bank/tag2idx.json', 'r') as inp:
    tag2idx = json.load(inp)
    idx2tag = {i: w for w, i in tag2idx.items()}

In [8]:
def inference(texts, bsz=32):
    texts = [clean_text(x.strip()) for x in texts]
    texts = np.array([[word2char.get(w, word2char['<UNK>']) for w in sent] for sent in texts])
    results = []
    for chars, lens, char_lens in get_batch_classifier_inference(texts, bsz):
        res = session.run([classifier.probs, tagger.decode_sequence], feed_dict={
            language_model.inputs: chars, language_model.seq_lens: lens,
            language_model.char_lens: char_lens, language_model.bptt: 20
        })
        results.append(res)
    return results

In [10]:
inference(['Dịch vụ 4d secure là gì vậy'])

[[array([[2.6383090e-10, 4.1220969e-07, 2.5529323e-09, 4.4460057e-06,
          1.3986375e-10, 4.0912549e-09, 9.9999332e-01, 1.7166545e-06,
          1.2660480e-07, 1.8831827e-09, 1.0862791e-09, 4.7357842e-09,
          2.1025070e-08, 3.9238029e-10, 3.7833159e-09, 3.3428937e-09,
          1.2676154e-08, 1.2719356e-09, 4.1492596e-09]], dtype=float32),
  array([[0, 0, 0, 2, 2, 0, 0, 0, 0]], dtype=int32)]]